#Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import classification_report,f1_score, confusion_matrix, accuracy_score

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/swuds/t5_thesis/98_progress/dataset/nd_sales_LogisticRegression_base.csv')
df.head()

#Preprocess

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

In [ ]:
df['lag_1'] = df['sales_sum'].shift(1)
df['lag_2'] = df['sales_sum'].shift(2)
df['lag_3'] = df['sales_sum'].shift(3)
df['lag_4'] = df['sales_sum'].shift(4)
df['lag_5'] = df['sales_sum'].shift(5)
df['lag_6'] = df['sales_sum'].shift(6)
df['lag_7'] = df['sales_sum'].shift(7)
df['lag_8'] = df['sales_sum'].shift(8)
df['lag_9'] = df['sales_sum'].shift(9)
df['lag_10'] = df['sales_sum'].shift(10)
df['lag_11'] = df['sales_sum'].shift(11)

In [ ]:
df.dropna(inplace=True)
df.head()

In [ ]:
df_train_window = df.tail(187)
df_train_window

In [ ]:
threshold = np.percentile(df_train_window['sales_sum'], 66)
threshold

In [ ]:
df_train_window['is_hotseller'] = np.where(df_train_window['sales_sum'] >= threshold, 1, 0)
df_train_window.head()

In [ ]:
# นับจำนวน is_hot_seller ที่เป็น 1 และ 0
is_hotseller_counts = df_train_window['is_hotseller'].value_counts()

print("จำนวน is_hotseller ที่เป็น 1:", is_hotseller_counts.get(1, 0))
print("จำนวน is_hotseller ที่เป็น 0:", is_hotseller_counts.get(0, 0))

In [ ]:
df_train_window.tail(8)

In [ ]:
df_train_window = df_train_window.reindex(columns=['is_hotseller', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11'])

In [ ]:
test_data = df_train_window.tail(7)
train_data = df_train_window.head(180)

print("Test Data:")
print(test_data)

print("\nTrain Data:")
print(train_data)

In [ ]:
X = train_data[['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11']]
y = train_data['is_hotseller']

#Train

มีความจำเป็นต้องกำหนด class_weight='balanced' เนื่องจากปัญหา Imbalance target ซึ่งเกิดจากการต้องการแยกวันสำคัญ ซึ่งคือวันขายดีกว่าปกติ ซึ่งเป็นส่วนน้อย

In [ ]:
# เพิ่ม class_weight='balanced' เข้าไป
#model = LogisticRegression(class_weight='balanced')
model = LogisticRegression()
model.fit(X, y)

In [ ]:
X_test = test_data[['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11']]
predictions = model.predict(X_test)

# Print the predictions
print("\nPredictions:")
predictions

#Evaluate

In [ ]:
y_test = test_data['is_hotseller']

In [ ]:
# คำนวณ Confusion Matrix
cm = confusion_matrix(y_test, predictions)

# แสดง Confusion Matrix
print("\nConfusion Matrix:")
print(cm)

In [ ]:
print("\nClassification Report:")
print(classification_report(y_test, predictions))